## Import Library

In [3]:
# Data Handling
import pandas as pd
import datetime as dt
import random
import itertools
# from selenium import webdriver
# from selenium.webdriver.common.by import By
import gc

# Data Visualizing
from tqdm import tqdm

#model
from sklearn.model_selection import train_test_split

# etc.
import warnings
warnings.filterwarnings('ignore')
import os
import time

DATA_PATH = 'C:/Users/USER/Desktop/Bbeojung.kr/Data'
CHROME_DRIVER = os.path.join(DATA_PATH, 'chromedriver_win32/chromedriver.exe')

## Download DataSet

In [27]:
# # Download df
# driver = webdriver.Chrome(CHROME_DRIVER)
# url = 'https://bigdata.gwangju.go.kr/usr/main/goMain.rd'
# driver.get(url)

# transp_btn = driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div/div[3]/div/ul/li[18]/a')
# transp_btn.click()

# time.sleep(1)
# page_btn = driver.find_element(By.XPATH, '//*[@id="6"]')
# page_btn.click()

# for page in range(7,24):
#     for i in range(1,11):
#         time.sleep(1)
#         view_btn = driver.find_element(By.XPATH, f'//*[@id="dataSetListBody"]/tr[{i}]/td[8]/a')
#         view_btn.click()
#         while True:
#             try:        
#                 download_btn = driver.find_element(By.XPATH, '//*[@id="apiSrvTbTop"]/tbody/tr[6]/td[1]/a[1]')
#                 download_btn.click()                
#                 tolist_btn = driver.find_element(By.XPATH, '//*[@id="mainForm"]/div[2]/input')
#                 tolist_btn.click()
#             except:
#                 continue
#             break                

#     if page % 10 == 1:
#         time.sleep(1)
#         next_btn = driver.find_element(By.XPATH, f'//*[@id="next"]')
#         next_btn.click()

#     time.sleep(1)
#     page_btn = driver.find_element(By.XPATH, f'//*[@id="{page}"]')
#     page_btn.click()

# driver.quit()


In [28]:
# def csv_to_parquet(file_name):
#     read_path = os.path.join(DATA_PATH, f'train_data_csv/{file_name}.csv')
#     save_path = os.path.join(DATA_PATH, f'train_data_parquet/{file_name}.parquet')

#     df = pd.read_csv(read_path, encoding='cp949')
#     df.to_parquet(save_path)
#     del df
#     gc.collect()
#     print(file_name, 'Done.')


# path = os.path.join(DATA_PATH, 'train_data_csv')
# for (root, dirs, files) in os.walk(path):
#     for file in files:
#         file_name, _ = os.path.splitext(file)
#         csv_to_parquet(file_name)

## Load DataSet

In [1]:
target_name = 'num'
origin_time = ['ATTEND', 'WKDAY_NOON', 'LVFFC', 'RESTDE_NOON']
origin_how = ['TKON', 'GFF']
origin_col = [
'BUSSTAT_BSNM_ID', 'BUSSTAT_BSNM_ROUTEID', 'month', 'weekday', 

'BUSSTAT_TKON_ALL_ATTEND', 'BUSSTAT_TKON_ALL_WKDAY_NOON',
'BUSSTAT_TKON_ALL_LVFFC', 'BUSSTAT_TKON_ALL_RESTDE_NOON',             

'BUSSTAT_GFF_ALL_ATTEND', 'BUSSTAT_GFF_ALL_WKDAY_NOON',
'BUSSTAT_GFF_ALL_LVFFC', 'BUSSTAT_GFF_ALL_RESTDE_NOON',                    
]
new_time = ['work', 'wdnoon', 'quit', 'wenoon']
new_how = ['tkon', 'tkoff']
new_col = [
    'bstop', 'bus', 'month', 'weekday',

    'tkon_work', 'tkon_wdnoon', 'tkon_quit', 'tkon_wenoon',

    'tkoff_work', 'tkoff_wdnoon', 'tkoff_quit', 'tkoff_wenoon',
]

def id_statistic(train, test, ID, col1, col2):
    # train mean, sum
    train_mean = train.groupby([ID])[target_name].agg([(col1, 'mean')]).reset_index()
    train_sum = train.groupby([ID])[target_name].agg([(col2, 'sum')]).reset_index()
    train_mean_sum = pd.merge(train_mean, train_sum, on=ID)

    # merge
    train_res = pd.merge(train, train_mean_sum, how='left', on=ID)
    test_res = pd.merge(test, train_mean_sum, how='left', on=ID)
    
    return train_res, test_res         


def make_target(df, col):
    how, when = col.split('_')
    
    res_df = pd.DataFrame({
        'how' : [how] * len(df[col]),
        'when' : [when] * len(df[col]),
        'num' : df[col]
    })

    return pd.concat([df.iloc[:,:4], res_df], axis=1)

def makeCol(train, test,  col_list, i):
    datas = list(itertools.combinations(col_list, i))
    for data in datas:
        col = []
        train_tmp=train[data[0]]
        test_tmp=test[data[0]]
        col.append(data[0])

        for j in range(1,i):
            col.append(data[j])
            train_tmp = train_tmp + '_' + train[ data[j] ]
            test_tmp = test_tmp + '_' + test[ data[j] ]

        col = '_'.join(col)
        train[col] = train_tmp
        test[col] = test_tmp

        train, test = id_statistic(train, test, col, 'mean_'+col, 'sum_'+col)  

    del col
    del train_tmp
    del test_tmp

    return train, test

def make_dateset(df, how):
    # # TKON = TKCAR + TRNSIT
    # for a in origin_age:
    #     for t in origin_time:
    #         df[f'BUSSTAT_TKON_{a}_{t}'] = df[f'BUSSTAT_TKCAR_{a}_{t}'] + df[f'BUSSTAT_TRNSIT_{a}_{t}']
    for t in origin_time:
        df[f'BUSSTAT_TKON_ALL_{t}'] = df[f'BUSSTAT_TKCAR_ALL_{t}'] + df[f'BUSSTAT_TRNSIT_ALL_{t}']

    # # MINOR = YNGBGS + CHILD
    # for h in origin_how:
    #     for t in origin_time:
    #         df[f'BUSSTAT_{h}_MINOR_{t}'] = df[f'BUSSTAT_{h}_YNGBGS_{t}'] + df[f'BUSSTAT_{h}_CHILD_{t}']

    df['BUSSTAT_BSNM_ID'] = df['BUSSTAT_BSNM_ID'].astype(str)
    df['BUSSTAT_BSNM_ROUTEID'] = df['BUSSTAT_BSNM_ROUTEID'].astype(str)
    date_col = pd.to_datetime(df.BUSSTAT_DT, format='%Y%m%d')
    df['month'] = date_col.apply(lambda x: x.month)
    df['weekday'] = date_col.dt.weekday

    # Select Columns
    df = df[origin_col]
    df.columns = new_col

    # make target
    data = pd.DataFrame()
    for t in new_time:
        for h in new_how:
            data = pd.concat([data, make_target(df, f'{h}_{t}')])

    # select col / split
    col = [
        'bstop', 'bus', 'month', 'weekday',
        'how', 'when', 'num'
    ]        
    train, test = train_test_split(data[col], test_size=0.3, shuffle=True, random_state=123)            

    # select how
    train = train[train.how == how]
    test = test[test.how == how]

    # feature engineering
    col_list = ['bstop', 'bus', 'month', 'weekday', 'when']
    for col in col_list:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
    

    for i in range(1,6):
        train, test = makeCol(train, test, col_list, i)

    return train, test

In [4]:
df = pd.read_parquet( 'C:/Users/USER/Desktop/Bbeojung.kr/Data/train_data_parquet/2022년 03월 07일 교통카드 이용통계.parquet' )
train, test = make_dateset( df, 'tkon' )

In [5]:
test.isna().sum()

bstop                                    0
bus                                      0
month                                    0
weekday                                  0
how                                      0
                                     ...  
mean_bus_month_weekday_when              0
sum_bus_month_weekday_when               0
bstop_bus_month_weekday_when             0
mean_bstop_bus_month_weekday_when    10728
sum_bstop_bus_month_weekday_when     10728
Length: 95, dtype: int64

In [14]:
train = pd.DataFrame()
test = pd.DataFrame()
data_path = os.path.join(DATA_PATH, 'train_data_parquet')
sample_num = 2000

for (root, dirs, files) in os.walk(data_path):
    for file in tqdm(files):
        tmp_df = pd.read_parquet( os.path.join(data_path, file) )
        tmp_df = tmp_df.iloc[random.sample(tmp_df.index.tolist(), sample_num if sample_num != 0 else len(tmp_df)),:]

        train_tmp, test_tmp = make_dateset( tmp_df, 'tkon' )
        train = pd.concat([train, train_tmp])
        test = pd.concat([test, test_tmp])

        del tmp_df, train_tmp, test_tmp
        gc.collect()

train.to_parquet(os.path.join(DATA_PATH, f'final_dataset/train_{sample_num}.parquet'))
test.to_parquet(os.path.join(DATA_PATH, f'final_dataset/test_{sample_num}.parquet'))        

100%|██████████| 220/220 [05:31<00:00,  1.50s/it]
